### Import Dependencies

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

from config import conn_string
#from config_testdb import conn_string

**get all files for loading**

In [ ]:
# path
station_path = os.path.join('.', "station_info.csv")
weather_path = os.path.join('.', "historical_weather.csv")
pricing_path = os.path.join('.', "pricing_index.csv")

#bikes
toronto_bikes = os.path.join("Resources/1_toronto_canada", "toronto_bikes.csv")
vancouver_bikes = os.path.join("Resources/2_vancouver_canada", "vancouver_bikes.csv")
boston_bikes = os.path.join("Resources/3_boston_us", "boston_bikes.csv")
nyc_bikes = os.path.join("Resources/4_NewYork_US", "nyc_bikes.csv")

# membertypes
toronto_users = os.path.join("Resources/1_toronto_canada", "toronto_membertypes.csv")
vancouver_users = os.path.join("Resources/2_vancouver_canada", "vancouver_membertypes.csv")
boston_users = os.path.join("Resources/3_boston_us", "boston_membertypes.csv")
nyc_users = os.path.join("Resources/4_NewYork_US", "nyc_membertypes.csv")

# ridership
toronto_ = os.path.join("Resources/1_toronto_canada", "toronto_bikeshare.csv")
vancouver_ = os.path.join("Resources/2_vancouver_canada", "vancouver_bikeshare.csv")
boston_ = os.path.join("Resources/3_boston_us", "boston_bikeshare.csv")
nyc_ = os.path.join("Resources/4_NewYork_US", "nyc_bikeshare.csv")


**read all files into dataframes**

In [ ]:
# read files
stns = pd.read_csv(station_path, encoding="utf-8")
weather = pd.read_csv(weather_path, encoding="utf-8")

# bikes
tor_bikes = pd.read_csv(toronto_bikes, encoding="utf-8")
van_bikes = pd.read_csv(vancouver_bikes, encoding="utf-8")
bos_bikes = pd.read_csv(boston_bikes, encoding="utf-8")
nyc_bikes = pd.read_csv(nyc_bikes, encoding="utf-8")

# member_types
tor_users = pd.read_csv(toronto_users, encoding="utf-8")
van_users = pd.read_csv(vancouver_users, encoding="utf-8")
bos_users = pd.read_csv(boston_users, encoding="utf-8")
nyc_users = pd.read_csv(nyc_users, encoding="utf-8")

# pricing index
pricing_index = pd.read_csv(pricing_path, encoding="utf-8")

# ridership
toronto = pd.read_csv(toronto_, encoding="utf-8")
vancouver = pd.read_csv(vancouver_, encoding="utf-8")
boston = pd.read_csv(boston_, encoding="utf-8")
newyork = pd.read_csv(nyc_, encoding="utf-8")


**Locations**

In [ ]:
# create locations table
locations = [{"location_id": 1,"city":"Toronto", "country":"Canada" }, 
             {"location_id": 2,"city":"Vancouver", "country":"Canada"},
             {"location_id": 3,"city":"Boston", "country":"USA"},
             {"location_id": 4,"city":"New York", "country":"USA"}
            ]

df = pd.DataFrame(locations)
df.set_index("location_id", inplace=True)
df


**Stations**

In [ ]:
stns.head()

**Historical Weather**

In [ ]:
weather.head()

**Bikes**

In [ ]:
tor_bikes.head()

In [ ]:
van_bikes.head()

In [ ]:
bos_bikes.head()

In [ ]:
nyc_bikes.head()

**Member Types**

In [ ]:
tor_users.head()

In [ ]:
van_users.head()

In [ ]:
bos_users.head()

In [ ]:
nyc_users.head()

**Pricing Index**

In [ ]:
pricing_index.head()

**Ridership**

In [ ]:
# Toronto
toronto.head()

In [ ]:
# Vancouver
vancouver.head()

In [ ]:
# Boston
boston.head()

In [ ]:
# New York
newyork.head()

### Load data

#### Connect to database

In [ ]:
# connect to database
cnx = f'postgresql://{conn_string}'
engine = create_engine(cnx)

In [ ]:
# confirm tables
engine.table_names()

***Locations***

In [ ]:
# load locations into database
df.to_sql(name='locations', con=engine, if_exists='append', index=True)
print("locations")

***Stations***

In [ ]:
# load stations into database
stns.to_sql(name='stations', con=engine, if_exists='append', index=False)
print("stations")

***Historical Weather***

In [ ]:
# load historical weather into database
weather.to_sql(name='historicalweather', con=engine, if_exists='append', index=False)
print("weather")

***Bikes***

In [ ]:
# load toronto bikes into database
tor_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)
print("tor_bikes")

In [ ]:
# load vancouver bikes into database
van_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)
print("van_bikes")

In [ ]:
# load boston bikes into database
bos_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)
print("bos_bikes")

In [ ]:
# load new york bikes into database
nyc_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)
print("nyc_bikes")

***Member Types***

In [ ]:
# load toronto member types
tor_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)
print("tor_users")

In [ ]:
# load vancouver member types
van_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)
print("van_users")

In [ ]:
# load boston member types
bos_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)
print("bos_users")

In [ ]:
# load new york member types
nyc_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)
print("nyc_users")

***Pricing Index***

In [ ]:
# load pricing index
pricing_index.to_sql(name='pricing', con=engine, if_exists='append',index=False, index_label='price_id')
print("pricing_index")

### Load Ridership

#### check reference to stations table

In [ ]:
# create session
session = Session(bind=engine)
# declare a base using automap_base
Base = automap_base()
# use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
Base.classes.keys()
Stations = Base.classes.stations

In [ ]:
Base.classes.keys()

***Verify that all station_ids used in bikeshare exists in Stations table***

In [ ]:
# get all stations
all_stations = session.query(Stations.station_id).all()
all_stns_df = pd.DataFrame(all_stations)
all_stns_df

### ***Toronto***

In [ ]:
# check for Toronto records with start_station_id not in stations table
missing_start = toronto[-toronto["start_station_id"].isin(all_stns_df["station_id"])]
missing_start = missing_start.rename(columns={"start_station_id":"station_id"})

In [ ]:
missing_start

In [ ]:
# check for Toronto records with end_station_id not in stations table
missing_end = toronto[-toronto["end_station_id"].isin(all_stns_df["station_id"])]
missing_end = missing_end.rename(columns={"start_station_id":"station_id"})


In [ ]:
missing_end

*Load rows with station_id existing in stations table*

In [ ]:
toronto = toronto[toronto["start_station_id"].isin(all_stns_df["station_id"])]
toronto = toronto[toronto["end_station_id"].isin(all_stns_df["station_id"])]

In [ ]:
toronto

In [ ]:
# get all Toronto stations
stns_toronto = session.query(Stations.station_id).filter(Stations.location_id==1).all()
stns_toronto = pd.DataFrame(stns_toronto)
stns_toronto = stns_toronto.rename(columns={"station_id": "start_station_id"})
stns_toronto

In [ ]:
# remove Toronto records with start_station_id not in stations table
tor_df = pd.merge(toronto, stns_toronto, how='inner', on='start_station_id')

In [ ]:
tor_df

In [ ]:
# rename Toronto station column to check if end_station_id exists in stations table
stns_toronto = stns_toronto.rename(columns={"start_station_id": "end_station_id"})
stns_toronto

In [ ]:
#remove Toronto records with end_station_id not in stations table
toronto = pd.merge(tor_df, stns_toronto, how='inner', on='end_station_id')

In [ ]:
# count number of records for insert to ridership table
print(toronto.count())

In [ ]:
# Load Toronto bikeshare data to ridership table
toronto.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

### ***Vancouver***

In [ ]:
# check for Vancouver records with start_station_id missing in stations table
missing_start = vancouver[-vancouver["start_station_id"].isin(all_stns_df["station_id"])]
missing_start = missing_start.rename(columns={"start_station_id":"station_id"})


In [ ]:
missing_start

In [ ]:
# check for Vancouver records with end_station_id missing in stations table
missing_end = vancouver[-vancouver["end_station_id"].isin(all_stns_df["station_id"])]
missing_end = missing_end.rename(columns={"start_station_id":"station_id"})


In [ ]:
missing_end

In [ ]:
# check Vancouver records with start_station_id and end_station_id existing in stations table
vancouver = vancouver[vancouver["start_station_id"].isin(all_stns_df["station_id"])]
vancouver = vancouver[vancouver["end_station_id"].isin(all_stns_df["station_id"])]

In [ ]:
vancouver

In [ ]:
# get all Vancouver stations
stns_vancouver = session.query(Stations.station_id).filter(Stations.location_id==2).all()
stns_vancouver = pd.DataFrame(stns_vancouver)
stns_vancouver = stns_vancouver.rename(columns={"station_id": "start_station_id"})
stns_vancouver

In [ ]:
# remove Vancouver records with start_station_id not in stations table
vancouver_df = pd.merge(vancouver, stns_vancouver, how='inner', on='start_station_id')

In [ ]:
# rename Vancouver station column to end_station_id
stns_vancouver = stns_vancouver.rename(columns={"start_station_id": "end_station_id"})
stns_vancouver.head()

In [ ]:
#remove Vancouver records with end_station_id not in stations table
vancouver = pd.merge(vancouver_df, stns_vancouver, how='inner', on='end_station_id')

In [ ]:
# check number of records for insert to database
vancouver.count()
print(vancouver.count())

In [ ]:
# Load Vancouver bikeshare data to ridership table
vancouver.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

### ***Boston***

In [ ]:
# check Boston data for missing start_station_id
missing_start = boston[-boston["start_station_id"].isin(all_stns_df["station_id"])]
missing_start

In [ ]:
# check for missing end_station_id
missing_end = boston[-boston["end_station_id"].isin(all_stns_df["station_id"])]
missing_end

In [ ]:
boston = boston[boston["start_station_id"].isin(all_stns_df["station_id"])]
boston = boston[boston["end_station_id"].isin(all_stns_df["station_id"])]
boston

In [ ]:
# get all Boston stations
stns_boston = session.query(Stations.station_id).filter(Stations.location_id==3).all()
stns_boston = pd.DataFrame(stns_boston)
stns_boston = stns_boston.rename(columns={"station_id": "start_station_id"})
stns_boston

In [ ]:
# remove Boston records with start_station_id not in stations table
df = pd.merge(boston, stns_boston, how='inner', on='start_station_id')

In [ ]:
# rename Boston station column to end_station_id
stns_boston = stns_boston.rename(columns={"start_station_id": "end_station_id"})
stns_boston.head()

In [ ]:
#remove Boston records with end_station_id not in stations table
boston_df = pd.merge(df, stns_boston, how='inner', on='end_station_id')

In [ ]:
# count number of Boston records for insert to ridership table
boston_df.count()
print(boston_df.count())

In [ ]:
# load Boston records to ridership table
boston_df.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

### ***New York***

In [ ]:
# check for New York records with missing start_station_id in stations table
missing_start = newyork[-newyork["start_station_id"].isin(all_stns_df["station_id"])]
missing_start

In [ ]:
# check for New York records with missing end_station_id in stations table
missing_end = newyork[-newyork["end_station_id"].isin(all_stns_df["station_id"])]
missing_end

In [ ]:
# check New York records with start_station_id and end_station_id both in stations table
newyork = newyork[newyork["start_station_id"].isin(all_stns_df["station_id"])]
newyork = newyork[newyork["end_station_id"].isin(all_stns_df["station_id"])]
newyork

In [ ]:
# get New York stations data
stns_newyork = session.query(Stations.station_id).filter(Stations.location_id==4).all()
stns_newyork = pd.DataFrame(stns_newyork)
stns_newyork = stns_newyork.rename(columns={"station_id": "start_station_id"})
stns_newyork

In [ ]:
# remove New York records with start_station_id not existing in stations table
ny_df = pd.merge(newyork, stns_newyork, how='inner', on='start_station_id')

In [ ]:
# rename New Yorks stations column to end_station_id
stns_newyork = stns_newyork.rename(columns={"start_station_id": "end_station_id"})
stns_newyork.head()

In [ ]:
# remove New York records with end_station_id not existing in stations table
newyork_df = pd.merge(ny_df, stns_newyork, how='inner', on='end_station_id')

In [ ]:
# count number of New York records for insert to ridership table
newyork_df.count()
print(newyork_df.count())

In [ ]:
newyork_df.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

**Close session**

In [ ]:
session.close()